In [1]:
import pandas as pd
import numpy as np
import itertools

In [149]:
# %%bash
# pwd
read_csv_flag = 3

if read_csv_flag == 1:
    original_df = pd.read_csv("../../../../data/real_application/foursquare_NY/fs_ny_top_users_10.csv",sep=";")
elif read_csv_flag == 2:
    original_df = pd.read_csv("../../../../data/real_application/foursquare_NY/fs_ny_top_users_81.csv",sep=";")
elif read_csv_flag == 3:
    original_df = pd.read_csv("../../../../data/real_application/foursquare_NY/fs_ny_top_users_193.csv",sep=";")
else:
    print('It not a valide option!')

In [150]:
original_df.head()
# original_df.info()

,new_tid,tid,lat_lon,time,day,poi,type,root_type,rating,weather,label
0,0,126,40.8331652006224 -73.9418603427692,317,Monday,The Lair Of Modern Strange Cowboy,Home (private),Residence,-1.0,Clear,6
1,0,126,40.8340978041072 -73.9452672225881,1404,Monday,Galaxy Gourmet Deli,Deli / Bodega,Food,8.2,Clouds,6
2,0,126,40.8331652006224 -73.9418603427692,0,Tuesday,The Lair Of Modern Strange Cowboy,Home (private),Residence,-1.0,Clouds,6
3,0,126,40.7646959283254 -73.8851974964414,1069,Thursday,Popeyes Louisiana Kitchen,Fried Chicken Joint,Food,6.6,Clear,6
4,0,126,40.7660790376824 -73.8835287094116,1120,Thursday,MTA Bus Operations Depot - LaGuardia,Bus Station,Travel & Transport,-1.0,Clear,6


In [151]:
format_time_output(1056)

'17:36'

In [152]:
def format_time_output(time_in_min):
    ''' 
    This function converts the minutes for the format Hours:Minutes.
    '''
    time_in_sec = float(time_in_min)*60
    hours = np.floor((time_in_sec/3600))
    mins = np.floor((time_in_sec - (hours*3600))/60)
    secs = np.floor(time_in_sec%60)
#     print(str(int(hours))+'h:'+str(int(mins))+'m:'+str(int(secs))+'s')
    return str(int(hours))+':'+str(int(mins))
#     return str(int(hours))+'h:'+str(int(mins))+'m:'+str(int(secs))+'s'

In [153]:
original_df['time_conv'] = original_df['time'].map(lambda x: format_time_output(x))
original_df.head()

,new_tid,tid,lat_lon,time,day,poi,type,root_type,rating,weather,label,time_conv
0,0,126,40.8331652006224 -73.9418603427692,317,Monday,The Lair Of Modern Strange Cowboy,Home (private),Residence,-1.0,Clear,6,5:17
1,0,126,40.8340978041072 -73.9452672225881,1404,Monday,Galaxy Gourmet Deli,Deli / Bodega,Food,8.2,Clouds,6,23:24
2,0,126,40.8331652006224 -73.9418603427692,0,Tuesday,The Lair Of Modern Strange Cowboy,Home (private),Residence,-1.0,Clouds,6,0:0
3,0,126,40.7646959283254 -73.8851974964414,1069,Thursday,Popeyes Louisiana Kitchen,Fried Chicken Joint,Food,6.6,Clear,6,17:49
4,0,126,40.7660790376824 -73.8835287094116,1120,Thursday,MTA Bus Operations Depot - LaGuardia,Bus Station,Travel & Transport,-1.0,Clear,6,18:40


In [154]:
work_df = original_df.copy()
work_df.drop(['new_tid','tid','lat_lon','poi','root_type','label','time_conv'], axis=1, inplace=True)
work_df.head()

,time,day,type,rating,weather
0,317,Monday,Home (private),-1.0,Clear
1,1404,Monday,Deli / Bodega,8.2,Clouds
2,0,Tuesday,Home (private),-1.0,Clouds
3,1069,Thursday,Fried Chicken Joint,6.6,Clear
4,1120,Thursday,Bus Station,-1.0,Clear


In [155]:
# list(range(0,360+1,30))
# np.arange(0, 360+1, 30)
def custom_bins_map(df_bin):
    start = ''
    end = ''
    
    df_bin = ''.join(str(df_bin).split(','))
    df_bin = ''.join(df_bin.split('('))
    df_bin = ''.join(df_bin.split(']'))
    df_bin = df_bin.split(' ')
    
    if df_bin[0] == '-0.001':
        start = '0:0'
        end = format_time_output(df_bin[1])
    else:
        start = format_time_output(str(df_bin[0]))
        end = format_time_output(df_bin[1])
    
    return '('+start+', '+end+']'

In [156]:
custom_bins_map('(1050.0, 1080.0]')

'(17:30, 18:0]'

In [157]:
bins = np.arange(0, 1440+1, 30)
work_df['time_bins'] = pd.cut(work_df.time, bins=bins, include_lowest=True)
work_df['time_bins'] = work_df['time_bins'].map(lambda x: custom_bins_map(x))
work_df['time_discrete_label'] = pd.cut(work_df.time, bins=4, labels=['dawn','morning','afternoon','night'], include_lowest=True)
# work_df['time_bins'] = pd.qcut(work_df.time, q=48)
work_df.head()

,time,day,type,rating,weather,time_bins,time_discrete_label
0,317,Monday,Home (private),-1.0,Clear,"(5:0, 5:30]",dawn
1,1404,Monday,Deli / Bodega,8.2,Clouds,"(23:0, 23:30]",night
2,0,Tuesday,Home (private),-1.0,Clouds,"(0:0, 0:30]",dawn
3,1069,Thursday,Fried Chicken Joint,6.6,Clear,"(17:30, 18:0]",afternoon
4,1120,Thursday,Bus Station,-1.0,Clear,"(18:30, 19:0]",night


In [158]:
work_df[work_df['time']==0].head()

,time,day,type,rating,weather,time_bins,time_discrete_label
2,0,Tuesday,Home (private),-1.0,Clouds,"(0:0, 0:30]",dawn
80,0,Tuesday,Home (private),-1.0,Clouds,"(0:0, 0:30]",dawn
1226,0,Sunday,Home (private),-1.0,Clouds,"(0:0, 0:30]",dawn
2294,0,Saturday,Home (private),-1.0,Clear,"(0:0, 0:30]",dawn
10883,0,Saturday,Airport Terminal,-1.0,Clear,"(0:0, 0:30]",dawn


In [159]:
print(work_df.time.min())
print(work_df.time.max())
print(work_df.time_bins.nunique())
work_df.drop('time',axis=1,inplace=True)

0
1439
48


In [160]:
# work_df['rating']=work_df['rating'].astype('object')
# work_df['time_bins']=work_df['time_bins'].astype('object')
# work_df = work_df.convert_dtypes()
# work_df = work_df.astype('category')
work_df = work_df.applymap(str)
work_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66946 entries, 0 to 66945
Data columns (total 6 columns):
day                    66946 non-null object
type                   66946 non-null object
rating                 66946 non-null object
weather                66946 non-null object
time_bins              66946 non-null object
time_discrete_label    66946 non-null object
dtypes: object(6)
memory usage: 3.1+ MB


In [161]:
### Reordering
columns = list(work_df.columns)
print(columns)
t = columns.copy()
columns[0],columns[1],columns[2],columns[3],columns[4],columns[5] = t[1],t[0],t[5],t[4],t[3],t[2]
print(columns)
del t
work_df = work_df[columns]
work_df.head()

['day', 'type', 'rating', 'weather', 'time_bins', 'time_discrete_label']
['type', 'day', 'time_discrete_label', 'time_bins', 'weather', 'rating']


,type,day,time_discrete_label,time_bins,weather,rating
0,Home (private),Monday,dawn,"(5:0, 5:30]",Clear,-1.0
1,Deli / Bodega,Monday,night,"(23:0, 23:30]",Clouds,8.2
2,Home (private),Tuesday,dawn,"(0:0, 0:30]",Clouds,-1.0
3,Fried Chicken Joint,Thursday,afternoon,"(17:30, 18:0]",Clear,6.6
4,Bus Station,Thursday,night,"(18:30, 19:0]",Clear,-1.0


In [162]:
# stuff = columns
# from itertools import chain, combinations
# def all_subsets(ss):
#     return chain(*map(lambda x: combinations(ss, x), range(0, len(ss)+1)))

# for subset in all_subsets(stuff):
#     print(subset)

In [163]:
# import itertools

# for L in range(1, len(stuff)+1):
#     print(L)
#     for subset in itertools.combinations(stuff, L):
#         print(subset)

Function to concatenate the dimensions of a dataframe

In [164]:
def create_dimension_concatenation(df,dimensions):
    '''Method designed for up to 6 dimensions'''
#     columns = list(df.columns)
    print('Number of initial columns: ',len(dimensions))
    for L in range(2, len(dimensions)+1):
        ### skip L=1 because it is the original single dimension
#         print(L)
        for subset in itertools.combinations(dimensions, L):
#             print(subset)
            if L == 2:
#                 print(subset[0],subset[1])
                df[subset[0]+'|'+subset[1]] = df[subset[0]]+'|'+df[subset[1]]
            elif L == 3:
#                 print(subset[0],subset[1],subset[2])
                df[subset[0]+'|'+subset[1]+'|'+subset[2]] = df[subset[0]] +'|'+ df[subset[1]] +'|'+ df[subset[2]]
            elif L == 4:
#                 print(subset[0],subset[1],subset[2],subset[3])
                df[subset[0]+'|'+subset[1]+'|'+subset[2]+'|'+subset[3]] = df[subset[0]]+'|'+df[subset[1]]+'|'+df[subset[2]]+'|'+df[subset[3]]
            elif L == 5:
#                 print(subset[0],subset[1],subset[2],subset[3],subset[4])
                df[subset[0]+'|'+subset[1]+'|'+subset[2]+'|'+subset[3]+'|'+subset[4]] = df[subset[0]]+'|'+df[subset[1]]+'|'+df[subset[2]]+'|'+df[subset[3]]+'|'+df[subset[4]]
            elif L == 6:
#                 print(subset[0],subset[1],subset[2],subset[3],subset[4])
                df[subset[0]+'|'+subset[1]+'|'+subset[2]+'|'+subset[3]+'|'+subset[4]+'|'+subset[5]] = df[subset[0]]+'|'+df[subset[1]]+'|'+df[subset[2]]+'|'+df[subset[3]]+'|'+df[subset[4]]+'|'+df[subset[5]]
            else:
                print('Method designed for up to 6 dimensions.\n It has more than the limite.')
                break
    return df

In [165]:
concatenated_work_df = create_dimension_concatenation(work_df.copy(),columns)
print('Number of final columns: ',len(list(concatenated_work_df.columns)))
concatenated_work_df.head()

Number of initial columns:  6
Number of final columns:  63


,type,day,time_discrete_label,time_bins,weather,rating,type|day,type|time_discrete_label,type|time_bins,type|weather,...,day|time_discrete_label|weather|rating,day|time_bins|weather|rating,time_discrete_label|time_bins|weather|rating,type|day|time_discrete_label|time_bins|weather,type|day|time_discrete_label|time_bins|rating,type|day|time_discrete_label|weather|rating,type|day|time_bins|weather|rating,type|time_discrete_label|time_bins|weather|rating,day|time_discrete_label|time_bins|weather|rating,type|day|time_discrete_label|time_bins|weather|rating
0,Home (private),Monday,dawn,"(5:0, 5:30]",Clear,-1.0,Home (private)|Monday,Home (private)|dawn,"Home (private)|(5:0, 5:30]",Home (private)|Clear,...,Monday|dawn|Clear|-1.0,"Monday|(5:0, 5:30]|Clear|-1.0","dawn|(5:0, 5:30]|Clear|-1.0","Home (private)|Monday|dawn|(5:0, 5:30]|Clear","Home (private)|Monday|dawn|(5:0, 5:30]|-1.0",Home (private)|Monday|dawn|Clear|-1.0,"Home (private)|Monday|(5:0, 5:30]|Clear|-1.0","Home (private)|dawn|(5:0, 5:30]|Clear|-1.0","Monday|dawn|(5:0, 5:30]|Clear|-1.0","Home (private)|Monday|dawn|(5:0, 5:30]|Clear|-1.0"
1,Deli / Bodega,Monday,night,"(23:0, 23:30]",Clouds,8.2,Deli / Bodega|Monday,Deli / Bodega|night,"Deli / Bodega|(23:0, 23:30]",Deli / Bodega|Clouds,...,Monday|night|Clouds|8.2,"Monday|(23:0, 23:30]|Clouds|8.2","night|(23:0, 23:30]|Clouds|8.2","Deli / Bodega|Monday|night|(23:0, 23:30]|Clouds","Deli / Bodega|Monday|night|(23:0, 23:30]|8.2",Deli / Bodega|Monday|night|Clouds|8.2,"Deli / Bodega|Monday|(23:0, 23:30]|Clouds|8.2","Deli / Bodega|night|(23:0, 23:30]|Clouds|8.2","Monday|night|(23:0, 23:30]|Clouds|8.2","Deli / Bodega|Monday|night|(23:0, 23:30]|Cloud..."
2,Home (private),Tuesday,dawn,"(0:0, 0:30]",Clouds,-1.0,Home (private)|Tuesday,Home (private)|dawn,"Home (private)|(0:0, 0:30]",Home (private)|Clouds,...,Tuesday|dawn|Clouds|-1.0,"Tuesday|(0:0, 0:30]|Clouds|-1.0","dawn|(0:0, 0:30]|Clouds|-1.0","Home (private)|Tuesday|dawn|(0:0, 0:30]|Clouds","Home (private)|Tuesday|dawn|(0:0, 0:30]|-1.0",Home (private)|Tuesday|dawn|Clouds|-1.0,"Home (private)|Tuesday|(0:0, 0:30]|Clouds|-1.0","Home (private)|dawn|(0:0, 0:30]|Clouds|-1.0","Tuesday|dawn|(0:0, 0:30]|Clouds|-1.0","Home (private)|Tuesday|dawn|(0:0, 0:30]|Clouds..."
3,Fried Chicken Joint,Thursday,afternoon,"(17:30, 18:0]",Clear,6.6,Fried Chicken Joint|Thursday,Fried Chicken Joint|afternoon,"Fried Chicken Joint|(17:30, 18:0]",Fried Chicken Joint|Clear,...,Thursday|afternoon|Clear|6.6,"Thursday|(17:30, 18:0]|Clear|6.6","afternoon|(17:30, 18:0]|Clear|6.6","Fried Chicken Joint|Thursday|afternoon|(17:30,...","Fried Chicken Joint|Thursday|afternoon|(17:30,...",Fried Chicken Joint|Thursday|afternoon|Clear|6.6,"Fried Chicken Joint|Thursday|(17:30, 18:0]|Cle...","Fried Chicken Joint|afternoon|(17:30, 18:0]|Cl...","Thursday|afternoon|(17:30, 18:0]|Clear|6.6","Fried Chicken Joint|Thursday|afternoon|(17:30,..."
4,Bus Station,Thursday,night,"(18:30, 19:0]",Clear,-1.0,Bus Station|Thursday,Bus Station|night,"Bus Station|(18:30, 19:0]",Bus Station|Clear,...,Thursday|night|Clear|-1.0,"Thursday|(18:30, 19:0]|Clear|-1.0","night|(18:30, 19:0]|Clear|-1.0","Bus Station|Thursday|night|(18:30, 19:0]|Clear","Bus Station|Thursday|night|(18:30, 19:0]|-1.0",Bus Station|Thursday|night|Clear|-1.0,"Bus Station|Thursday|(18:30, 19:0]|Clear|-1.0","Bus Station|night|(18:30, 19:0]|Clear|-1.0","Thursday|night|(18:30, 19:0]|Clear|-1.0","Bus Station|Thursday|night|(18:30, 19:0]|Clear..."


In [166]:
print(concatenated_work_df.columns)

Index(['type', 'day', 'time_discrete_label', 'time_bins', 'weather', 'rating',
       'type|day', 'type|time_discrete_label', 'type|time_bins',
       'type|weather', 'type|rating', 'day|time_discrete_label',
       'day|time_bins', 'day|weather', 'day|rating',
       'time_discrete_label|time_bins', 'time_discrete_label|weather',
       'time_discrete_label|rating', 'time_bins|weather', 'time_bins|rating',
       'weather|rating', 'type|day|time_discrete_label', 'type|day|time_bins',
       'type|day|weather', 'type|day|rating',
       'type|time_discrete_label|time_bins',
       'type|time_discrete_label|weather', 'type|time_discrete_label|rating',
       'type|time_bins|weather', 'type|time_bins|rating',
       'type|weather|rating', 'day|time_discrete_label|time_bins',
       'day|time_discrete_label|weather', 'day|time_discrete_label|rating',
       'day|time_bins|weather', 'day|time_bins|rating', 'day|weather|rating',
       'time_discrete_label|time_bins|weather',
       'time_d

In [167]:
# ['new_tid','tid','lat_lon','poi','root_type','label']
concatenated_work_df.insert(0,'new_tid',original_df['new_tid'])
concatenated_work_df.insert(1,'lat_lon',original_df['lat_lon'])
concatenated_work_df.insert(2,'poi',original_df['poi'])
concatenated_work_df.insert(3,'root_type',original_df['root_type'])
concatenated_work_df.insert(4,'label',original_df['label'])
concatenated_work_df.insert(7,'time',original_df['time'])
concatenated_work_df.insert(8,'time_conv',original_df['time_conv'])
# print(concatenated_work_df.columns)
# print(concatenated_work_df.head())

Index(['new_tid', 'lat_lon', 'poi', 'root_type', 'label', 'type', 'day',
       'time', 'time_conv', 'time_discrete_label', 'time_bins', 'weather',
       'rating', 'type|day', 'type|time_discrete_label', 'type|time_bins',
       'type|weather', 'type|rating', 'day|time_discrete_label',
       'day|time_bins', 'day|weather', 'day|rating',
       'time_discrete_label|time_bins', 'time_discrete_label|weather',
       'time_discrete_label|rating', 'time_bins|weather', 'time_bins|rating',
       'weather|rating', 'type|day|time_discrete_label', 'type|day|time_bins',
       'type|day|weather', 'type|day|rating',
       'type|time_discrete_label|time_bins',
       'type|time_discrete_label|weather', 'type|time_discrete_label|rating',
       'type|time_bins|weather', 'type|time_bins|rating',
       'type|weather|rating', 'day|time_discrete_label|time_bins',
       'day|time_discrete_label|weather', 'day|time_discrete_label|rating',
       'day|time_bins|weather', 'day|time_bins|rating', 'day|

In [168]:
if read_csv_flag == 1:
    print('Creating concat dimenstions file for top 10 users.')
    concatenated_work_df.to_csv('fs_ny_t10_concat.csv', encoding='utf-8', sep=';', index=False)
elif read_csv_flag == 2:
    print('Creating concat dimenstions file for top 81 users.')
    concatenated_work_df.to_csv('fs_ny_t81_concat.csv', encoding='utf-8', sep=';', index=False)
elif read_csv_flag == 3:
    print('Creating concat dimenstions file for top 193 users.')
    concatenated_work_df.to_csv('fs_ny_t193_concat.csv', encoding='utf-8', sep=';', index=False)
else:
    print('It not a valide option!')

Creating concat dimenstions file for top 193 users.
